# Introduction

Authors: Scardecchia Mattia, Filatrella Dario
Date: 07/06/2023

For our project, we decided to study the phase transition of the ferromagnetic Potts model on a square lattice.

The Potts model is a generalization of the Ising model and consists of a system of interacting spins that can be in either of $q$ states. Spins are located on a two-dimensional square lattice of length $L$, and neighbouring spins interact according to the Hamiltonian: $$ H(s_1, ..., s_N) = - J \sum_{(ij)} \delta (s_i, s_j) ,$$
where the summation is over all pairs of neighbouring spins on the lattice (with periodic boundary conditions). $J$ is a positive constant and it represents the strength of the ferromagnetic interaction.

Using the statistical physics approach, we can study the maximally entropic Boltzmann distribution $$ p(s_1, ..., s_N) = \frac{1}{Z(\beta)} \exp \left( - \beta H(s_1, ..., s_N) \right) ,$$
where $\beta$ is the inverse temperature and $Z(\beta)$ is the partition function and it can be used to derive all the thermodynamic averages of the system, if an analytical expression is available.

It is well known (https://arxiv.org/abs/1611.09877 ; https://arxiv.org/abs/1505.04159) that this model exhibits a phase transition in the thermodynamic limit ($N \to \infty$) at: $$ \beta J = \log \left( 1 + \sqrt{q} \right) .$$ The transition is continuous for $q \le 4$, and discontinuous for $q > 4$.
These results were obtained analytically. For our project, we want to use Monte Carlo methods, and in particular the Metropolis algorithm, to compute thermodynamic averages of the system and explore the phase transition.

In order to do this, we computed the average energy, magnetization and specific heat of the system at different temperatures, and we plotted the results to examine the behaviour around the theoretical critical temperature. Since the phase transition exists in the thermodynamic limit, we used increasing values of $L$ to detect potential finite-size effects.

The main challenge was the necessity to simulate a large system many times to explore the dependency of the thermodynamic averages on temperature, which is computationally very expensive. To overcome this challenge, we had to resort to a series of optimizations that made our code run orders of magnitude faster than a naive Python implementation.

As we will show, our numerical results pin down the critical temperature to a good accuracy, and already with small lattice sizes  we do observe a phase transition, which is significantly sharper for $q > 4$ than for $q \le 4$.

- Theoretical status
- phase transitions
- goals and methods

Phase transitions. Theoretical results. Goals and methods.

# Monte Carlo Simulations

q = 4. high and low temperature. close and far from critical temperature. plot stuff.